# Monitor Azure AI Services
Azure AI Services can be a critical part of an overall application infrastructure. It’s important to be able to monitor activity and get alerted to issues that may need attention.

## Provision an Azure AI Services resource
If you don’t already have one in your subscription, you’ll need to provision an Azure AI Services resource.

## Configure an alert
Let’s start monitoring by defining an alert rule so you can detect activity in your Azure AI services resource.

1. In the Azure portal, go to your Azure AI services resource and view its Alerts page (in the Monitoring section).
2. Select + Create dropdown, and select Alert rule
3. In the Create an alert rule page, under Scope, verify that the your Azure AI services resource is listed. (Close Select a signal pane if open)
4. Select Condition tab, and click on the See all signals link to show the Select a signal pane that appears on the right, where you can select a signal type to monitor.
5. In the Signal type list, select Activity Log, and then in the filtered list, select List Keys.
6. Review the activity over the past 6 hours.
7. Select the Actions tab. Note that you can specify an action group. This enables you to configure automated actions when an alert is fired - for example, sending an email notification. We won’t do that in this exercise; but it can be useful to do this in a production environment.
8. In the Details tab, set the Alert rule name to Key List Alert.
9. Select Review + create.
10. Review the configuration for the alert. Select Create and wait for the alert rule to be created.
11. Enter the following command to sign into your Azure subscription by using the Azure CLI.

In [13]:
!az login

A web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.
[
  {
    "cloudName": "AzureCloud",
    "homeTenantId": "9f29c0ee-e759-4225-8eb9-8ad87ca0802e",
    "id": "037c30a5-5ce7-43c3-b964-703eeb6b0dd7",
    "isDefault": true,
    "managedByTenants": [],
    "name": "Azure subscription 1",
    "state": "Enabled",
    "tenantId": "9f29c0ee-e759-4225-8eb9-8ad87ca0802e",
    "user": {
      "name": "nomantanveer021@gmail.com",
      "type": "user"
    }
  }
]


In [14]:
# This will create an alert in the Azure portal
!az cognitiveservices account keys list --name cogntive-srvc-2 --resource-group demo

{
  "key1": "c4e32354854548679076355bef22fa19",
  "key2": "2df24400b1fe490488a3313bc9eebba9"
}


- Switch back to the browser containing the Azure portal, and refresh your Alerts page. You should see a `Key List Alert` alert listed in the table (if not, wait up to five minutes and refresh again).
- Select the alert to see its details.

## Visualize a metric
As well as defining alerts, you can view metrics for your Azure AI services resource to monitor its utilization.

1. In the Azure portal, in the page for your Azure AI services resource, select Metrics (in the Monitoring section).
2. If there is no existing chart, select + New chart. Then in the Metric list, review the possible metrics you can visualize and select Total Calls.
3. In the Aggregation list, select Count. This will enable you to monitor the total calls to you Azure AI Service resource; which is useful in determining how much the service is being used over a period of time.
4. To generate some requests to your Azure AI service, you will use curl - a command line tool for HTTP requests. In Visual Studio Code, in the 03-monitor folder, open rest-test.cmd and edit the curl command it contains (shown below), replacing <yourEndpoint> and <yourKey> with your endpoint URI and Key1 key to use the Text Analytics API in your Azure AI services resource.

In [15]:
!curl -X POST "https://cogntive-srvc-2.cognitiveservices.azure.com/text/analytics/v3.1/languages?" -H "Content-Type: application/json" -H "Ocp-Apim-Subscription-Key: c4e32354854548679076355bef22fa19" --data-ascii "{'documents':[{'id':1,'text':'hello'}]}"

{"documents":[{"id":"1","detectedLanguage":{"name":"English","iso6391Name":"en","confidenceScore":1.0},"warnings":[]}],"errors":[],"modelVersion":"2022-10-01"}

The command returns a JSON document containing information about the language detected in the input data (which should be English).

1. Re-run the **rest-test** command multiple times to generate some call activity.
2. Return to the **Metrics** page in the Azure portal and refresh the **Total Calls** count chart. It may take a few minutes for the calls you made using curl to be reflected in the chart - keep refreshing the chart until it updates to include them.

One of the options for monitoring Azure AI services is to use diagnostic logging. Once enabled, diagnostic logging captures rich information about your Azure AI services resource usage, and can be a useful monitoring and debugging tool. It can take over an hour after setting up diagnostic logging to generate any information, which is why we haven’t explored it in this exercise; but you can learn more about it in the [Azure AI Services documentation](https://docs.microsoft.com/azure/cognitive-services/diagnostic-logging).

# Choose a service to containerize
Ensure you have Congitive AI Service configured.

## Deploy and run a Text Analytics container
In the Azure portal, on the Home page, select the ＋Create a resource button, search for container instances, and create a Container Instances resource with the following settings:

- Basics:
    - Subscription: Your Azure subscription
    - Resource group: Choose the resource group containing your Azure AI services resource
    - Container name: Enter a unique name
    - Region: Choose any available region
    - Image source: Other Registry
    - Image type: Public
    - Image: mcr.microsoft.com/azure-cognitive-services/textanalytics/language:latest
    - OS type: Linux
    - Size: 1 vcpu, 12 GB memory
- Networking:
    - Networking type: Public
    - DNS name label: Enter a unique name for the container endpoint
    - Ports: Change the TCP port from 80 to 5000
- Advanced:
    - Restart policy: On failure
    - Environment variables:
        Yes	ApiKey	Either key for your Azure AI services resource
        Yes	Billing	The endpoint URI for your Azure AI services resource
        No	Eula	accept
    - Command override: [ ]
- Tags:
    - Don’t add any tags
2. Wait for deployment to complete, and then go to the deployed resource.
3. Observe the following properties of your container instance resource on its Overview page:
    - Status: This should be Running.
    - IP Address: This is the public IP address you can use to access your container instances.
    - FQDN: This is the fully-qualified domain name of the container instances resource, you can use this to access the container instances instead of the IP address.

### Use the container
Run the cell below replacing <your_ACI_IP_address_or_FQDN> with the IP address or FQDN for your container.

In [16]:
!curl POST "http://ms-container.g9a3gze2dvaqfpa9.centralindia.azurecontainer.io:5000/text/analytics/v3.0/languages" -H "Content-Type: application/json" --data-ascii "{'documents':[{'id':1,'text':'Hello world.'},{'id':2,'text':'Salut tout le monde.'}]}"

curl: (6) Could not resolve host: POST
curl: (6) Could not resolve host: ms-container.g9a3gze2dvaqfpa9.centralindia.azurecontainer.io
